In [ ]:
from selenium import webdriver #1,2,3,,페이지가 동적 페이지이므로 셀레늄 사용.
import datetime, time
import requests
from bs4 import BeautifulSoup


chromedriver = './chromedriver'
driver = webdriver.Chrome(chromedriver)
driver.get('http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105') #네이버 it뉴스 홈페이지
#driver.get('http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105#&date=2016-10-19 00:00:00&page=1')

pagenumlist=[] #클릭시 나타나는 뉴스리스트의 페이지 담기
pagenum_1 = driver.find_element_by_css_selector('div#paging').text
if len(pagenum_1) > 1:
    print "len > 1"
    pagenumlist.append("1")
    pagenum = driver.find_elements_by_css_selector('a._paging')
    for pagenumcount in pagenum:
        pagenumlist.append(pagenumcount.text) #page가 여러 개일 경우
else:
    pagenumlist.append(pagenum_1) #page가 1밖에 없는 경우

print pagenum_1

hreflist=[] #각각 뉴스 링크 담기

for i in range(len(pagenumlist)):
    if i>0: #첫 페이지에서는 페이지 안넘기고 그대로 크롤링.
        link_page = driver.find_elements_by_css_selector('a._paging')
        link_page[i-1].click() #페이지 넘기기
        time.sleep(0.5) #넘기고 로드하는 시간 부여
    print pagenumlist[i]
    body = driver.find_element_by_id('section_body')
    a_body= body.find_elements_by_tag_name('a')
    for alist in a_body:
        #hreflist.append(alist.get_attribute("href"))

        if(alist.get_attribute("href").split('?')[1].split('=')[0] == 'mode'): #기사링크를 제외한 나머지 링크 불포함
            hreflist.append(alist.get_attribute("href"))# 뉴스 링크만 수집

print hreflist
driver.quit()

titlelist=[] #뉴스제목리스트
contentslist=[] #뉴스내용리스트

for link in hreflist:
    res = requests.get(link)
    soup = BeautifulSoup(res.text)
    tbody = soup.find("h3", attrs = {'id' : 'articleTitle'}) #뉴스제목탐색
    titlelist.append(tbody.text)
    cbody = soup.find("div", attrs = {'id' : 'articleBodyContents'}) #뉴스내용탐색
    contentslist.append(cbody.text)
    break
print titlelist[0]
    



In [27]:
import requests
from bs4 import BeautifulSoup

def news_crawler(url):
    news_url_list = []
    page_num = 1

    while True:
        response = requests.post('{}&mid=sec&mode=LSD&date=20161124&page={}'.format(url, page_num)) #itnews crawling
        soup = BeautifulSoup(response.content)

        links = soup.find_all('a', attrs={'class' : 'nclicks(fls.list)'}) # news links
        temp_length = len(news_url_list) # for stop [while]

        for link in links:
            if not link.attrs['href'] in news_url_list:
                print link
                news_url_list.append(link.attrs['href'])

        if temp_length == len(news_url_list):
            break # for stop [while]

        else:
            page_num += 1
    
    return news_url_list[0]

naver_itnews = 'http://news.naver.com/main/list.nhn?sid1=105'
print news_crawler(naver_itnews)


<a class="nclicks(fls.list)" href="http://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=105&amp;oid=030&amp;aid=0002554395">
<img alt="운전자가 제일 주의해야 할 국가, 프랑스-독일-이탈리아 순" src="http://imgnews.naver.net/image/thumb70/030/2016/11/24/2554395.jpg"/>
</a>
<a class="nclicks(fls.list)" href="http://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=105&amp;oid=092&amp;aid=0002106950">
<img alt="SKT, 적자투성이 SK컴즈 거두는 이유는" src="http://imgnews.naver.net/image/thumb70/092/2016/11/24/2106950.jpg"/>
</a>
<a class="nclicks(fls.list)" href="http://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=105&amp;oid=030&amp;aid=0002554393">
<img alt="인도 루피화 가치 역대 최저 수준으로 떨어져" src="http://imgnews.naver.net/image/thumb70/030/2016/11/24/2554393.jpg"/>
</a>
<a class="nclicks(fls.list)" href="http://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=105&amp;oid=022&amp;aid=0003119889">
								 O2O 자생논의… 머리 맞댄 카카오·국내업체
								</a>
<a class="nclicks(fls.list)" href="http://news.na

KeyboardInterrupt: 

In [105]:
import requests
from bs4 import BeautifulSoup
import re

def get_news_title(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
    
    title = soup.find('h3', attrs = {'id' : 'articleTitle'})
    company = soup.select(".press_logo > a > img ")[0].attrs['title']
    contents       = soup.find('div', attrs = {'id' : 'articleBodyContents'})
    date = soup.find('span', attr = {'class' : 't11'})
    return date.get_text()
    
    
url2 = 'http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=032&aid=0002745212'
#print get_news_title(url2)

res = requests.get(url2)
soup = BeautifulSoup(res.content)
print soup.select('.t11')[0].get_text()
#print re.sub(r'^\s*//.*\n.*{}\n+', '', get_news_title(url2))  
#print re.sub(r'^\s*//.*{}' ,'', '   //sswef  asef {} fea')
#print re.sub(r'{}' ,'', '   //sswef  asef {} fea')
#print re.search(r'([\w.-]+)@([\w.-]+)', get_news_title(url2)).group()
#email =  re.findall(r'[\w.-]+@[\w.-]+', 'ff@cp.kr   saas.kr')
#print email[-1]

2016-11-24 20:54


In [15]:
import redis
from exer_connection_db import server, redis_account

r= redis.Redis(host = server, port = 6379, password = redis_account['pw'])

r.delete('urllist')

0